In [4]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\ank10ki\Dropbox\phd\Image analysis\ColonyImages`


In [5]:
Threads.nthreads()

16

In [6]:
Pkg.status()

Project ColonyImages v1.0.0-DEV
Status `C:\Users\ank10ki\Dropbox\phd\Image analysis\ColonyImages\Project.toml`
⌃ [46ada45e] Agents v5.17.1
⌃ [6e4b80f9] BenchmarkTools v1.4.0
⌃ [336ed68f] CSV v0.10.12
⌃ [13f3f980] CairoMakie v0.11.5
⌅ [8be319e6] Chain v0.5.0
  [35d6a980] ColorSchemes v3.24.0
  [a93c6f00] DataFrames v1.6.1
⌃ [1313f7d8] DataFramesMeta v0.14.1
⌃ [864edb3b] DataStructures v0.18.16
  [31c24e10] Distributions v0.25.107
⌃ [e30172f5] Documenter v1.2.1
⌃ [634d3b9d] DrWatson v2.13.0
⌃ [7a1cc6ca] FFTW v1.7.2
⌃ [5789e2e9] FileIO v1.16.2
  [d8961e24] FindPeaks1D v0.1.8
⌃ [e9467ef8] GLMakie v0.9.5
⌅ [a09fc81d] ImageCore v0.9.4
  [2b14c160] ImageEdgeDetection v0.1.8
⌃ [6a3955dd] ImageFiltering v0.7.6
  [787d08f9] ImageMorphology v0.4.5
  [916415d5] Images v0.26.0
⌅ [a98d9a8b] Interpolations v0.14.7
  [682c06a0] JSON v0.21.4
  [2fda8390] LsqFit v0.15.0
  [e94cdb99] MosaicViews v0.3.4
  [d96e819e] Parameters v0.12.3
⌃ [295af30f] Revise v3.5.13
  [2913bbd2] StatsBase v0.34.2
  [ade2ca70]

In [7]:
using Revise, CairoMakie
using ColonyImages
using DataFrames, CSV, FileIO, Dates, Serialization
import ColorSchemes as co 

[ Info: Precompiling CairoMakie [13f3f980-e62b-5c42-98c6-ff1f3baf88f0]
Precompiling ColonyImages
  ✓ Git_jll
  ✓ libsixel_jll
  ✓ ImageMagick_jll
  ✓ OpenSSL
  ✓ XSLT_jll
  ✓ IntervalArithmetic → IntervalArithmeticDiffRulesExt
  ✓ RegionTrees
  ✓ CoordinateTransformations
  ✓ ArnoldiMethod
  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  ✓ Rotations
  ✓ FFTViews
  ✓ ForwardDiff
  ✓ Git
  ✓ StaticArrayInterface → StaticArrayInterfaceStaticArraysExt
  ✓ NearestNeighbors
  ✓ ForwardDiff → ForwardDiffStaticArraysExt
  ✓ LoopVectorization → SpecialFunctionsExt
  ✓ NLSolversBase
  ✓ HTTP
  ✓ Clustering
  ✓ Graphs
  ✓ LineSearches
  ✓ SimpleWeightedGraphs
  ✓ MetaGraphs
  ✓ StaticGraphs
  ✓ LoopVectorization → ForwardDiffExt
  ✓ MeshIO
  ✓ Optim
  ✓ LightOSM
  ✓ LsqFit
  ✓ Documenter
  ✓ Agents
  ✓ ImageCore
  ✓ JpegTurbo
  ✓ Sixel
  ✓ ImageBase
  ✓ ImageMagick
  ✓ PNGFiles
  ✓ ImageMorphology
  ✓ HistogramThresholding
  ✓ ImageAxes
  ✓ ImageDistances
  ✓ ImageBinarization
  ✓ ImageMetadata
  ✓ I

In [ ]:
df = DataFrame(data_set =String[], colony = String[], time = Int[], 
    metric_OG =Vector{Vector{Int64}}(undef,0), metric_cov = Vector{Vector{Int64}}(undef,0),
    pair_OG =Vector{Vector{Int64}}(undef,0),pair_cov =Vector{Vector{Int64}}(undef,0),OG_size = Int[])

In [ ]:
para = parameters(
        im_size = [800,800],
        time_points = [0:2.0:48...],
        colony_nr = 2);

simulations = ["Random", "Finger_weak", "Finger_strong"]
#simulations = ["Random"]
plot_sim = false

In [ ]:
name_of_sim = "$(para.colony_nr)_col_$(length(simulations))_grow_3200_"
fig_path = joinpath("data", "artificial",name_of_sim)
path_metric = joinpath("data", "metrics")
mkpath(path_metric)
mkpath(fig_path)

## Create empty datastructure for colony images

In [ ]:
pixel_to_add_vec = Int[];
vec_of_sims = Vector{Vector{BitArray{3}}}(undef, 0)
for i in 1:length(simulations)
    img_int_vec_1 = Vector{BitArray{3}}(undef, 0)
    for i in 1:para.colony_nr
        hans =BitArray(zeros(Bool, para.im_size...,para.stacks))
        push!(img_int_vec_1, hans)
    end
    push!(vec_of_sims, img_int_vec_1)
end

In [ ]:
pixel_to_add_vec = Int[];
@threads for (x,sim) in enumerate(simulations)
    for (i,colony) in enumerate(vec_of_sims[x])
        for (j,t) in enumerate(para.time_points)
            # use view to work on array directly 
            int_img  = @view colony[:,:,j]
            dir_vec = generate_dir_vec(para)
            if j == 1
                build_artifical_colony!(para.Center,int_img,para.radius_colony,para.Points)
                pixel_to_add_vec = para.pixel_to_add(int_img)
            else
                # for unkown reason views require [:] to make assignments 
                int_img[:] = colony[:,:,j-1]
                if sim == "Random"
                    expand_colony_radom_cov!(int_img,pixel_to_add_vec[j-1])
                elseif sim == "Finger_weak"
                    expand_colony_finger_radom_cov!(int_img,pixel_to_add_vec[j-1],dir_vec)
                elseif sim == "Finger_strong"
                    expand_colony_finger_radom_cov!(int_img,pixel_to_add_vec[j-1],dir_vec, still_spawn_rate= 0.0, dir_match_rate = 0.9995)
                end
            end
        end
    end   
end

### Plot first colony test wise to see if everything worked as intended

In [ ]:
fig_test = plot_time_series_cov_centroid(vec_of_sims[1][1:1], para)

###  Either plot sim into artifical data folder or not to save computation time

In [10]:
if plot_sim 
    @threads for (i,name) in enumerate(simulations)
        fig, data_set_name = plot_and_save_time_series_metrics!(vec_of_sims[i], para,df, name_data = name)
        save(joinpath(fig_path, data_set_name)*".png", fig)
    end
else
    @threads for (i,name) in enumerate(simulations)
        data_set_name = save_time_series_metrics!(vec_of_sims[i], para,df, name_data = name)
    end
end

In [11]:
first(df,2)

Row,data_set,colony,time,metric_OG,metric_cov,pair_OG,pair_cov,OG_size
,String,String,Int64,Array…,Array…,Array…,Array…,Int64
1,Random 2024_03_26,Colony 1 artifical,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0 … 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[21, 2, 16, 14, 13, 14, 15, 15, 13, 18 … 13, 18, 13, 15, 15, 14, 13, 14, 16, 21]","[5475, 5166, 5556, 5739, 5599, 5433, 5781, 5425, 5711, 5438 … 5558, 5612, 5389, 5646, 5387, 5663, 5806, 5682, 5038, 5887]","[5642, 5083, 5509, 5736, 5640, 5312, 5746, 5476, 5666, 5523 … 5489, 5590, 5504, 5765, 5332, 5517, 5672, 5625, 5158, 5971]",5249
2,Random 2024_03_26,Colony 1 artifical,2,"[2, 1, 0, 1, 0, 0, 0, 0, 0, 0 … 0, 0, 0, 0, 1, 2, 1, 1, 0, 2]","[0, 0, 0, 0, 1, 0, 1, 1, 1, 0 … 1, 0, 1, 1, 0, 2, 0, 2, 0, 0]","[4934, 6249, 10113, 9317, 3860, 9429, 4954, 6856, 6584, 6266 … 5546, 5531, 6111, 4250, 7275, 2387, 6464, 6261, 4187, 5880]","[3247, 5509, 10532, 6937, 2985, 10556, 3268, 6406, 6474, 5422 … 5099, 6541, 6353, 3982, 8734, 2711, 6430, 6585, 3975, 6114]",5249


In [12]:
serialize(joinpath(path_metric,"$(name_of_sim*Dates.format(now(), "yyyy_mm_dd")).jls"),df)
CSV.write(joinpath(path_metric,"$(name_of_sim*Dates.format(now(), "yyyy_mm_dd")).csv"),df)

"data\\metrics\\2_col_3_grow_3200_2024_03_26.csv"